In [1]:
import os
import sys
import importlib
from pathlib import Path

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

# %load_ext autoreload
# %autoreload 2

In [2]:
# MAIN DIR
main_dir = f'{str(Path(code_dir).parents[0])}/'

# code_dir
code_dir = f'{code_dir}/'
sys.path.append(code_dir)

# scraping dir
scraped_data = f'{code_dir}scraped_data/'

# data dir
data_dir = f'{code_dir}data/'

# lang models dir
llm_path = f'{data_dir}Language Models'

# sites
site_list=['Indeed', 'Glassdoor', 'LinkedIn']

# columns
cols=['Sector', 
      'Sector Code', 
      'Gender', 
      'Age', 
      'Language', 
      'Dutch Requirement', 
      'English Requirement', 
      'Gender_Female', 
      'Gender_Mixed', 
      'Gender_Male', 
      'Age_Older', 
      'Age_Mixed', 
      'Age_Younger', 
      'Gender_Num', 
      'Age_Num', 
      '% Female', 
      '% Male', 
      '% Older', 
      '% Younger']

int_variable: str = 'Job ID'
str_variable: str = 'Job Description'
gender: str = 'Gender'
age: str = 'Age'
language: str = 'en'
str_cols = ['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Job Description', 'Company URL', 'Job URL', 'Tracking ID']
nan_list = [None, 'None', '', ' ', [], -1, '-1', 0, '0', 'nan', np.nan, 'Nan']
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'


In [3]:
import string
import re
import time
import json
import csv
import glob
import pickle
import pandas as pd
import numpy as np
import googletrans
from googletrans import Translator
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.pipeline import Sentencizer


In [4]:
# This is a manually collected dictionary of incorrect/faulty keywords in scraped site data
with open(f'{scraped_data}CBS/Data/keyword_trans_dict.txt') as f:
    keyword_trans_dict = json.load(f)


In [5]:
# 111 words to fix
len(keyword_trans_dict)


111

In [6]:
def fix_broken_linkedin_files(glob_path):
    fix_list = []
    data_dict = {}
    data_list = []

    if glob_path.endswith('.json'):

        with open(glob_path, encoding = 'utf-8') as csv_file_handler:
            csv_reader = csv.DictReader(csv_file_handler)

            for rows in csv_reader:
                first_key = str(list(rows.keys())[0])
                key = rows[first_key]
                data_dict[key] = rows

        for num in data_dict:
            data_list.append(data_dict[num])

        with open(glob_path, 'w', encoding = 'utf-8') as json_file_handler:
            json_file_handler.write(json.dumps(data_list, indent = 4))
    
    return data_list


In [7]:
def fix_keywords(df_temp):
    if len(df_temp) > 0 and isinstance(df_temp, pd.DataFrame):
        for key, value in keyword_trans_dict.items():
            df_temp.loc[
                df_temp[df_temp['Search Keyword'].notnull()]['Search Keyword'].astype(str).apply(
                lambda x: x.lower().strip()
                ) == str(key).lower().strip(), 'Search Keyword'
            ] = str(value).lower().strip()

        unfixed = df_temp.loc[
            df_temp[df_temp['Search Keyword'].notnull()]['Search Keyword'].astype(str).apply(lambda x: x.lower().strip()).isin([x.lower().strip() for x in list(keyword_trans_dict.keys())])
        ]

        if len(unfixed) != 0:
            for key, value in keyword_trans_dict.items():
                for idx, row in df_temp.iterrows():
                    if row['Search Keyword'].astype(str).lower().strip() == str(key).lower().strip():
                        df_temp.loc[idx, 'Search Keyword'] = str(value).lower().strip()
    

    return df_temp


#### Read paths

In [8]:
glob_paths = []

for site in site_list:
    glob_paths.extend(glob.glob(f'{scraped_data}/{site}/Data/*.json')+glob.glob(f'{scraped_data}/{site}/Data/*.csv'))


In [9]:
# 955 json and csv files
len(glob_paths)


955

#### Use paths to open files, fix keywords, and drop unneeded columns

In [10]:
# Fix list catches all incorrect/faculty keyword search terms

fix_list = []

# Appended data catches all the fixed and cleaned dfs
appended_data = []

for glob_path in glob_paths:

    if glob_path.endswith('.json'):
        try:
            df_temp = pd.read_json(glob_path).reset_index(drop=True)
        except ValueError:
            fix_list.append(glob_path)
            if 'scraped_data/LinkedIn/Data/linkedin_jobs_df_' in glob_path:
                data_json = fix_broken_linkedin_files(glob_path)
                try:
                    df_temp = pd.read_json(glob_path).reset_index(drop=True)
                except ValueError:
                    fix_list.append(glob_path)
    elif glob_path.endswith('.csv'):
        df_temp = pd.read_csv(glob_path).reset_index(drop=True)

    if len(df_temp) > 0 and isinstance(df_temp, pd.DataFrame):
        df_temp = fix_keywords(df_temp)
        df_temp.reset_index(drop=True, inplace=True)
        df_temp.drop(columns=cols, axis='columns', inplace=True, errors='ignore')
        df_temp.drop(
        df_temp.columns[
                df_temp.columns.str.contains(
                    'unnamed|index|level', regex=True, case=False, flags=re.I
                )
            ],
            axis='columns',
            inplace=True,
            errors='ignore',
        )
    
        if glob_path.endswith('.json'):
            df_temp.to_json(glob_path, orient='records')
        elif glob_path.endswith('.csv'):
            df_temp.to_csv(glob_path, index=False)

        appended_data.append(df_temp.reset_index(drop=True))

# # Concatonate list of dfs into one large df_jobs
# df_jobs = pd.concat(appended_data).reset_index(drop=True)

# # Save df_jobs to file
# if len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame):
#     df_jobs.to_pickle(f'{data_dir}df_jobs_raw.pkl')
    
#     df_jobs.to_csv(f'{data_dir}df_jobs_raw.csv', index=False)
# else:
#     print(f'ERORR: LENGTH OF DF = {len(df_jobs)}')


In [11]:
# List of dfs, len = 527
len(appended_data)


527

In [12]:
# Concatonate list of dfs into one large df_jobs
df_jobs = pd.concat(appended_data).reset_index(drop=True)


In [13]:
# len = 204113
len(df_jobs)


204113

In [14]:
# Save df_jobs to file
if len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame):
    df_jobs.to_pickle(f'{data_dir}df_jobs_raw.pkl')
    
    df_jobs.to_csv(f'{data_dir}df_jobs_raw.csv', index=False)
else:
    print(f'ERORR: LENGTH OF DF = {len(df_jobs)}')


In [15]:
# If we couldn't fix some keywords, we add them to list fix_list and write to file
if len(fix_list) != 0:
    print('Some keywords to fix!')
    with open(f'{data_dir}fix_list.txt', 'w') as f:
        json.dump(fix_list, f)


### START HERE IF SOURCING FROM DF_RAW
### PLEASE SET CORRECT DIRECTORY PATHS BELOW

In [16]:
import os
import sys
import importlib
from pathlib import Path

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

# %load_ext autoreload
# %autoreload 2

In [17]:
# MAIN DIR
main_dir = f'{str(Path(code_dir).parents[0])}/'

# code_dir
code_dir = f'{code_dir}/'
sys.path.append(code_dir)

# scraping dir
scraped_data = f'{code_dir}scraped_data/'

# data dir
data_dir = f'{code_dir}data/'

# lang models dir
llm_path = f'{data_dir}Language Models'

# sites
site_list=['Indeed', 'Glassdoor', 'LinkedIn']

# columns
cols=['Sector', 
      'Sector Code', 
      'Gender', 
      'Age', 
      'Language', 
      'Dutch Requirement', 
      'English Requirement', 
      'Gender_Female', 
      'Gender_Mixed', 
      'Gender_Male', 
      'Age_Older', 
      'Age_Mixed', 
      'Age_Younger', 
      'Gender_Num', 
      'Age_Num', 
      '% Female', 
      '% Male', 
      '% Older', 
      '% Younger']

int_variable: str = 'Job ID'
str_variable: str = 'Job Description'
gender: str = 'Gender'
age: str = 'Age'
language: str = 'en'
str_cols = ['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Job Description', 'Company URL', 'Job URL', 'Tracking ID']
nan_list = [None, 'None', '', ' ', [], -1, '-1', 0, '0', 'nan', np.nan, 'Nan']
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'


In [18]:
import string
import re
import time
import json
import csv
import glob
import pickle
import pandas as pd
import numpy as np
import googletrans
from googletrans import Translator
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.pipeline import Sentencizer


In [19]:
df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw.pkl').reset_index(drop=True)


In [20]:
# len = 204113
len(df_jobs)


204113

In [21]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204113 entries, 0 to 204112
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Search Keyword     204113 non-null  object 
 1   Platform           204113 non-null  object 
 2   Job ID             204113 non-null  object 
 3   Job Title          204113 non-null  object 
 4   Company Name       204103 non-null  object 
 5   Location           204113 non-null  object 
 6   Job Description    204098 non-null  object 
 7   Rating             51158 non-null   float64
 8   Employment Type    203053 non-null  object 
 9   Company URL        193659 non-null  object 
 10  Job URL            204113 non-null  object 
 11  Job Age            204113 non-null  object 
 12  Job Age Number     204113 non-null  object 
 13  Collection Date    204113 non-null  object 
 14  Data Row           152949 non-null  float64
 15  Tracking ID        152949 non-null  object 
 16  In

In [22]:
df_jobs.columns

Index(['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name',
       'Location', 'Job Description', 'Rating', 'Employment Type',
       'Company URL', 'Job URL', 'Job Age', 'Job Age Number',
       'Collection Date', 'Data Row', 'Tracking ID', 'Industry', 'Job Date',
       'Type of ownership'],
      dtype='object')

In [23]:
# Clean columns
df_jobs.columns = df_jobs.columns.to_series().apply(lambda x: str(x).strip())


In [24]:
# Drop NA
df_jobs.dropna(axis='index', how='all', inplace=True)
df_jobs.dropna(axis='columns', how='all', inplace=True)


In [25]:
# len = 204113
len(df_jobs)


204113

In [26]:
# Drop duplicates on subset of 'Job Description'
df_jobs.drop_duplicates(subset=['Job Description'], keep='first', ignore_index=True, inplace=True)


In [27]:
# len = 62579
len(df_jobs)


62579

In [28]:
# Remove any rows with missing 'Job Description'
df_jobs.drop(
    df_jobs[
        (df_jobs['Job Description'].isin(nan_list)) | 
        (df_jobs['Job Description'].isnull()) | 
        (df_jobs['Job Description'].isna())
    ].index, 
    axis='index',
    inplace=True,
    errors='ignore'

)


In [29]:
# len = 62577
len(df_jobs)


62577

In [30]:
# Save df_jobs to file
if len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame):
    df_jobs.to_pickle(f'{data_dir}df_jobs_raw_dropped.pkl')
    
    df_jobs.to_csv(f'{data_dir}df_jobs_raw_dropped.csv', index=False)
else:
    print(f'ERORR: LENGTH OF DF = {len(df_jobs)}')


### START HERE IF SOURCING FROM DF_RAW_DROPPED
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [31]:
import os
import sys
import importlib
from pathlib import Path

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

# %load_ext autoreload
# %autoreload 2

In [32]:
# MAIN DIR
main_dir = f'{str(Path(code_dir).parents[0])}/'

# code_dir
code_dir = f'{code_dir}/'
sys.path.append(code_dir)

# scraping dir
scraped_data = f'{code_dir}scraped_data/'

# data dir
data_dir = f'{code_dir}data/'

# lang models dir
llm_path = f'{data_dir}Language Models'

# sites
site_list=['Indeed', 'Glassdoor', 'LinkedIn']

# columns
cols=['Sector', 
      'Sector Code', 
      'Gender', 
      'Age', 
      'Language', 
      'Dutch Requirement', 
      'English Requirement', 
      'Gender_Female', 
      'Gender_Mixed', 
      'Gender_Male', 
      'Age_Older', 
      'Age_Mixed', 
      'Age_Younger', 
      'Gender_Num', 
      'Age_Num', 
      '% Female', 
      '% Male', 
      '% Older', 
      '% Younger']

int_variable: str = 'Job ID'
str_variable: str = 'Job Description'
gender: str = 'Gender'
age: str = 'Age'
language: str = 'en'
str_cols = ['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Job Description', 'Company URL', 'Job URL', 'Tracking ID']
nan_list = [None, 'None', '', ' ', [], -1, '-1', 0, '0', 'nan', np.nan, 'Nan']
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'


In [33]:
import string
import re
import time
import json
import csv
import glob
import pickle
import pandas as pd
import numpy as np
import googletrans
from googletrans import Translator
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.pipeline import Sentencizer


In [34]:
df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_dropped.pkl').reset_index(drop=True)


In [35]:
# 62577
len(df_jobs)


62577

In [36]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62577 entries, 0 to 62576
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Search Keyword     62577 non-null  object 
 1   Platform           62577 non-null  object 
 2   Job ID             62577 non-null  object 
 3   Job Title          62577 non-null  object 
 4   Company Name       62574 non-null  object 
 5   Location           62577 non-null  object 
 6   Job Description    62577 non-null  object 
 7   Rating             3975 non-null   float64
 8   Employment Type    61995 non-null  object 
 9   Company URL        59263 non-null  object 
 10  Job URL            62577 non-null  object 
 11  Job Age            62577 non-null  object 
 12  Job Age Number     62577 non-null  object 
 13  Collection Date    62577 non-null  object 
 14  Data Row           58599 non-null  float64
 15  Tracking ID        58599 non-null  object 
 16  Industry           591

In [37]:
# df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_language_detected.pkl').reset_index(drop=True)


In [38]:
# 62577
len(df_jobs)


62577

In [39]:
# df_jobs['Language'].value_counts()

In [ ]:
translator = Translator()
googletrans_readtime_error = googletrans.client.httpx._client.httpcore._exceptions.ReadTimeout

if 'Language' not in df_jobs.columns:
    df_jobs['Language'] = np.nan # create Language col and fill it with nan

# try:
#     time.sleep(60)
#     df_jobs['Language'] = df_jobs['Job Description'].apply(lambda x: translator.detect(str(x).lower().strip()).lang)
# except:
#     time.sleep(3600)
#     df_jobs['Language'] = df_jobs['Job Description'].apply(lambda x: translator.detect(str(x).lower().strip()).lang)

for idx, row in df_jobs.iterrows():
    # This part ensures we don't start lang detection from index 0 if some lang detection was already done
    if len(str(row['Job Description'])) != 0:
        if type(row['Language']) == float and np.isnan(row['Language']): #if lang is nan, detect language

            try:
                print(f'Row {idx}: Translation in progress.')
#                 time.sleep(10)
                df_jobs.loc[idx, 'Language'] = str(translator.detect(str(row['Job Description']).lower().strip()).lang)
                print(f'Row {idx}: Translation done.')
            except:# googletrans_readtime_error:

                if len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame):
                    df_jobs.to_pickle(f'{data_dir}df_jobs_raw_language_detected.pkl')

                    df_jobs.to_csv(f'{data_dir}df_jobs_raw_language_detected.csv', index=False)
                else:
                    print(f'ERORR: LENGTH OF DF = {len(df_jobs)}')

                print(f'Row {idx}: Sleeping for half an hour starting at {time.strftime("%I:%M:%S %p", time.localtime())}.')
                print('-'*30)
                time.sleep(1800)
                print(f'Row {idx}: Done sleeping.')
                print('-'*30)
                df_jobs.loc[idx, 'Language'] = str(translator.detect(str(row['Job Description']).lower().strip()).lang)

        else: # elif lang is not nan, skip and go to next idx
            continue

if len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame):
    df_jobs.to_pickle(f'{data_dir}df_jobs_raw_language_detected.pkl')

    df_jobs.to_csv(f'{data_dir}df_jobs_raw_language_detected.csv', index=False)
else:
    print(f'ERORR: LENGTH OF DF = {len(df_jobs)}')


Row 234: Translation in progress.
Row 234: Translation done.
Row 235: Translation in progress.
Row 235: Translation done.
Row 236: Translation in progress.
Row 236: Translation done.
Row 237: Translation in progress.
Row 237: Translation done.
Row 238: Translation in progress.
Row 238: Translation done.
Row 239: Translation in progress.
Row 239: Translation done.
Row 240: Translation in progress.
Row 240: Translation done.
Row 241: Translation in progress.
Row 241: Translation done.
Row 242: Translation in progress.
Row 242: Translation done.
Row 243: Translation in progress.
Row 243: Translation done.
Row 244: Translation in progress.
Row 244: Translation done.
Row 245: Translation in progress.
Row 245: Translation done.
Row 246: Translation in progress.
Row 246: Translation done.
Row 247: Translation in progress.
Row 247: Translation done.
Row 248: Translation in progress.
Row 248: Translation done.
Row 249: Translation in progress.
Row 249: Translation done.
Row 250: Translation in 

Row 370: Translation done.
Row 371: Translation in progress.
Row 371: Translation done.
Row 372: Translation in progress.
Row 372: Translation done.
Row 373: Translation in progress.
Row 373: Translation done.
Row 374: Translation in progress.
Row 374: Translation done.
Row 375: Translation in progress.
Row 375: Translation done.
Row 376: Translation in progress.
Row 376: Translation done.
Row 377: Translation in progress.
Row 377: Translation done.
Row 378: Translation in progress.
Row 378: Translation done.
Row 379: Translation in progress.
Row 379: Translation done.
Row 380: Translation in progress.
Row 380: Translation done.
Row 381: Translation in progress.
Row 381: Translation done.
Row 382: Translation in progress.
Row 382: Translation done.
Row 383: Translation in progress.
Row 383: Translation done.
Row 384: Translation in progress.
Row 384: Translation done.
Row 385: Translation in progress.
Row 385: Translation done.
Row 386: Translation in progress.
Row 386: Translation don

Row 507: Translation done.
Row 508: Translation in progress.
Row 508: Translation done.
Row 509: Translation in progress.
Row 509: Translation done.
Row 510: Translation in progress.
Row 510: Translation done.
Row 511: Translation in progress.
Row 511: Translation done.
Row 512: Translation in progress.
Row 512: Translation done.
Row 513: Translation in progress.
Row 513: Translation done.
Row 514: Translation in progress.
Row 514: Translation done.
Row 515: Translation in progress.
Row 515: Translation done.
Row 516: Translation in progress.
Row 516: Translation done.
Row 517: Translation in progress.
Row 517: Translation done.
Row 518: Translation in progress.
Row 518: Translation done.
Row 519: Translation in progress.
Row 519: Translation done.
Row 520: Translation in progress.
Row 520: Translation done.
Row 521: Translation in progress.
Row 521: Translation done.
Row 522: Translation in progress.
Row 522: Translation done.
Row 523: Translation in progress.
Row 523: Translation don

Row 645: Translation done.
Row 646: Translation in progress.
Row 646: Translation done.
Row 647: Translation in progress.
Row 647: Translation done.
Row 648: Translation in progress.
Row 648: Translation done.
Row 649: Translation in progress.
Row 649: Translation done.
Row 650: Translation in progress.
Row 650: Translation done.
Row 651: Translation in progress.
Row 651: Translation done.
Row 652: Translation in progress.
Row 652: Translation done.
Row 653: Translation in progress.
Row 653: Translation done.
Row 654: Translation in progress.
Row 654: Translation done.
Row 655: Translation in progress.
Row 655: Translation done.
Row 656: Translation in progress.
Row 656: Translation done.
Row 657: Translation in progress.
Row 657: Translation done.
Row 658: Translation in progress.
Row 658: Translation done.
Row 659: Translation in progress.
Row 659: Translation done.
Row 660: Translation in progress.
Row 660: Translation done.
Row 661: Translation in progress.
Row 661: Translation don

Row 781: Translation done.
Row 782: Translation in progress.
Row 782: Translation done.
Row 783: Translation in progress.
Row 783: Translation done.
Row 784: Translation in progress.
Row 784: Translation done.
Row 785: Translation in progress.
Row 785: Translation done.
Row 786: Translation in progress.
Row 786: Translation done.
Row 787: Translation in progress.
Row 787: Translation done.
Row 788: Translation in progress.
Row 788: Translation done.
Row 789: Translation in progress.
Row 789: Translation done.
Row 790: Translation in progress.
Row 790: Translation done.
Row 791: Translation in progress.
Row 791: Translation done.
Row 792: Translation in progress.
Row 792: Translation done.
Row 793: Translation in progress.
Row 793: Translation done.
Row 794: Translation in progress.
Row 794: Translation done.
Row 795: Translation in progress.
Row 795: Translation done.
Row 796: Translation in progress.
Row 796: Translation done.
Row 797: Translation in progress.
Row 797: Translation don

...

Row 12340: Translation done.
Row 12341: Translation in progress.
Row 12341: Translation done.
Row 12342: Translation in progress.
Row 12342: Translation done.
Row 12343: Translation in progress.
Row 12343: Translation done.
Row 12344: Translation in progress.
Row 12344: Translation done.
Row 12345: Translation in progress.
Row 12345: Translation done.
Row 12346: Translation in progress.
Row 12346: Translation done.
Row 12347: Translation in progress.
Row 12347: Translation done.
Row 12348: Translation in progress.
Row 12348: Translation done.
Row 12349: Translation in progress.
Row 12349: Translation done.
Row 12350: Translation in progress.
Row 12350: Translation done.
Row 12351: Translation in progress.
Row 12351: Translation done.
Row 12352: Translation in progress.
Row 12352: Translation done.
Row 12353: Translation in progress.
Row 12353: Translation done.
Row 12354: Translation in progress.
Row 12354: Translation done.
Row 12355: Translation in progress.
Row 12355: Translation do

Row 12468: Translation done.
Row 12469: Translation in progress.
Row 12469: Translation done.
Row 12470: Translation in progress.
Row 12470: Translation done.
Row 12471: Translation in progress.
Row 12471: Translation done.
Row 12472: Translation in progress.
Row 12472: Translation done.
Row 12473: Translation in progress.
Row 12473: Translation done.
Row 12474: Translation in progress.
Row 12474: Translation done.
Row 12475: Translation in progress.
Row 12475: Translation done.
Row 12476: Translation in progress.
Row 12476: Translation done.
Row 12477: Translation in progress.
Row 12477: Translation done.
Row 12478: Translation in progress.
Row 12478: Translation done.
Row 12479: Translation in progress.
Row 12479: Translation done.
Row 12480: Translation in progress.
Row 12480: Translation done.
Row 12481: Translation in progress.
Row 12481: Translation done.
Row 12482: Translation in progress.
Row 12482: Translation done.
Row 12483: Translation in progress.
Row 12483: Translation do

Row 12595: Translation done.
Row 12596: Translation in progress.
Row 12596: Translation done.
Row 12597: Translation in progress.
Row 12597: Translation done.
Row 12598: Translation in progress.
Row 12598: Translation done.
Row 12599: Translation in progress.
Row 12599: Translation done.
Row 12600: Translation in progress.
Row 12600: Translation done.
Row 12601: Translation in progress.
Row 12601: Translation done.
Row 12602: Translation in progress.
Row 12602: Translation done.
Row 12603: Translation in progress.
Row 12603: Translation done.
Row 12604: Translation in progress.
Row 12604: Translation done.
Row 12605: Translation in progress.
Row 12605: Translation done.
Row 12606: Translation in progress.
Row 12606: Translation done.
Row 12607: Translation in progress.
Row 12607: Translation done.
Row 12608: Translation in progress.
Row 12608: Translation done.
Row 12609: Translation in progress.
Row 12609: Translation done.
Row 12610: Translation in progress.
Row 12610: Translation do

Row 12722: Translation done.
Row 12723: Translation in progress.
Row 12723: Translation done.
Row 12724: Translation in progress.
Row 12724: Translation done.
Row 12725: Translation in progress.
Row 12725: Translation done.
Row 12726: Translation in progress.
Row 12726: Translation done.
Row 12727: Translation in progress.
Row 12727: Translation done.
Row 12728: Translation in progress.
Row 12728: Translation done.
Row 12729: Translation in progress.
Row 12729: Translation done.
Row 12730: Translation in progress.
Row 12730: Translation done.
Row 12731: Translation in progress.
Row 12731: Translation done.
Row 12732: Translation in progress.
Row 12732: Translation done.
Row 12733: Translation in progress.
Row 12733: Translation done.
Row 12734: Translation in progress.
Row 12734: Translation done.
Row 12735: Translation in progress.
Row 12735: Translation done.
Row 12736: Translation in progress.
Row 12736: Translation done.
Row 12737: Translation in progress.
Row 12737: Translation do

Row 12851: Translation done.
Row 12852: Translation in progress.
Row 12852: Translation done.
Row 12853: Translation in progress.
Row 12853: Translation done.
Row 12854: Translation in progress.
Row 12854: Translation done.
Row 12855: Translation in progress.
Row 12855: Translation done.
Row 12856: Translation in progress.
Row 12856: Translation done.
Row 12857: Translation in progress.
Row 12857: Translation done.
Row 12858: Translation in progress.
Row 12858: Translation done.
Row 12859: Translation in progress.
Row 12859: Translation done.
Row 12860: Translation in progress.
Row 12860: Translation done.
Row 12861: Translation in progress.
Row 12861: Translation done.
Row 12862: Translation in progress.
Row 12862: Translation done.
Row 12863: Translation in progress.
Row 12863: Translation done.
Row 12864: Translation in progress.
Row 12864: Translation done.
Row 12865: Translation in progress.
Row 12865: Translation done.
Row 12866: Translation in progress.
Row 12866: Translation do

In [ ]:
df_jobs['Language'].value_counts()

In [ ]:
# df_jobs.drop(
#     df_jobs[
#         (df_jobs['Language'] == "['en', 'nl']") | 
#         (df_jobs['Language'] == "['nl', 'en']")
#     ].index, 
#         axis='index', 
#         inplace=True, 
#         errors='ignore'
# )


In [ ]:
if len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame):
    df_jobs.to_pickle(f'{data_dir}df_jobs_raw_language_detected.pkl')
    
    df_jobs.to_csv(f'{data_dir}df_jobs_raw_language_detected.csv', index=False)
else:
    print(f'ERORR: LENGTH OF DF = {len(df_jobs)}')

In [ ]:
# Drop non-english job descriptions
# df_jobs.drop(
#     df_jobs[df_jobs['Language'] != str(language)].index, 
#     axis='index', 
#     inplace=True, 
#     errors='ignore'
# )


In [ ]:
# df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_language_detected.pkl').reset_index(drop=True)


In [ ]:
# Find and count unique search keywords
search_keywords = list(set(df_jobs['Search Keyword'].to_list()))


In [ ]:
len(search_keywords)


In [ ]:
search_keywords
